In [ ]:
# default_exp ml_utils

# ml_utils
> Utilities to use sklearn for image processing

In [ ]:
# export

from typing import Callable

import numpy as np
from numpy.core._multiarray_umath import ndarray
from sklearn.base import TransformerMixin, BaseEstimator


class LambdaRow(TransformerMixin, BaseEstimator):
    """
    
    Apply a function on each row of the numpy array
    
    """

    def __init__(self,
                 row_func: Callable[[ndarray], ndarray],
                 to_features: bool = False,
                 **kwargs):
        """Constructor for LambdaRow"""
        self.func = row_func
        self.func_kwargs = kwargs
        self.to_features = to_features

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        _arr = np.array([self.func(x, **self.func_kwargs) for x in X])
        if len(_arr.shape)>=2:
            if self.to_features:
                N = _arr.shape[0]
                return _arr.reshape(N,-1)
            else:
                return _arr
        else:
            return _arr.reshape(-1,1)

    def fit_transform(self, X, y=None, **fit_params):
        return self.fit(X).transform(X)

    def get_params(self, deep=True):
        return dict(list(self.func_kwargs.items())+[("row_func", self.func)])

    def set_params(self, **params):
        # params.pop("row_func")
        self.func_kwargs.update(params)
        return self

In [ ]:
import numpy as np

X = np.random.randint(0,255, size=(16,32,32))

LambdaRow(row_func=lambda x: x.mean()).fit_transform(X).shape

(16, 1)

## Training utils

In [ ]:
# export


import attr
from sklearn.pipeline import Pipeline
from typing import Callable, Dict, Any

@attr.s
class TrainingOutput(object):
    
    model = attr.ib(type=Pipeline)
    metrics = attr.ib(type=Dict[str, Any])
    

In [ ]:
# export

import numpy as np
from typing import Callable, Dict, Any
from sklearn.pipeline import Pipeline

def sk_train(
    xtrain: np.ndarray,
    xtest: np.ndarray,
    ytrain: np.ndarray,
    ytest: np.ndarray,
    model: Pipeline,
    metrics: Dict[str, Callable[[np.ndarray, np.ndarray], Any]]
    )->TrainingOutput:

    model.fit(xtrain, ytrain)

    computed_metrics = dict()

    for tag, (x,y) in [("train", (xtrain, ytrain) ), ("test", (xtest, ytest))]:
        predictions = model.predict(x)

        for metric_tag, metric_fn in metrics:
            computed_metrics["_".join([tag, metric_tag])] = metric_fn(y, predictions)
    
    return TrainingOutput(
        model=model,
        metrics=computed_metrics
    )
    

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X, Y = make_classification(10000)

xtrain, xtest, ytrain, ytest = train_test_split(X, Y, train_size=0.5)

sk_train(xtrain, xtest, ytrain, ytest, RandomForestClassifier(), metrics=dict(acc=accuracy_score).items())

/home/matthieu/anaconda2/envs/pypurr/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


TrainingOutput(model=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False), metrics={'train_acc': 0.9958, 'test_acc': 0.9596})

In [ ]:
# export

from sklearn.pipeline import Pipeline, make_pipeline

def make_single_feature_model(
    feature_fn: Callable[[np.ndarray], np.ndarray],
    clf: Pipeline
    )->Pipeline:

    return make_pipeline(
        LambdaRow(feature_fn),
        clf
    )